In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [2]:
X_train = np.load('processed/train.npz')['arr_0']
y_train = np.load('processed/train_y.npz')['arr_0']
X_test = np.load('processed/test.npz')['arr_0']
y_test = np.load('processed/test_y.npz')['arr_0']
X_valid = np.load('processed/valid.npz')['arr_0']
y_valid = np.load('processed/valid_y.npz')['arr_0']

In [3]:
X_train = X_train.reshape([X_train.shape[0], -1])
X_test = X_test.reshape([X_test.shape[0], -1])
X_valid = X_valid.reshape([X_valid.shape[0], -1])

In [4]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[8100]),
    layers.Dense(64, activation='relu'),
    layers.Dense(2)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [5]:
model = build_model()

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                518464    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
Total params: 522,754
Trainable params: 522,754
Non-trainable params: 0
_________________________________________________________________


In [7]:
EPOCHS = 1000

history = model.fit(
  X_train, y_train,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[tfdocs.modeling.EpochDots()])


Epoch: 0, loss:63.2905,  mae:2.7471,  mse:63.2904,  val_loss:1.7066,  val_mae:1.0153,  val_mse:1.7066,  
....................................................................................................
Epoch: 100, loss:1.0224,  mae:0.8673,  mse:1.0224,  val_loss:1.0679,  val_mae:0.8856,  val_mse:1.0679,  
....................................................................................................
Epoch: 200, loss:1.0224,  mae:0.8672,  mse:1.0224,  val_loss:1.0668,  val_mae:0.8854,  val_mse:1.0668,  
....................................................................................................
Epoch: 300, loss:1.0223,  mae:0.8674,  mse:1.0223,  val_loss:1.0674,  val_mae:0.8861,  val_mse:1.0674,  
....................................................................................................
Epoch: 400, loss:1.0223,  mae:0.8673,  mse:1.0223,  val_loss:1.0683,  val_mae:0.8865,  val_mse:1.0683,  
......................................................................